In [ ]:
!pip install roboticstoolbox-python
!pip install ipympl

In [ ]:
import roboticstoolbox as rtb
import spatialmath as sm
from math import pi, sin, cos
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget

In [ ]:
robot = rtb.DHRobot( 
    [ 
        rtb.RevoluteDH(a=1), 
        rtb.RevoluteDH(a=1) 
    ], 
    name="two_link")
print(robot)


Para nuestro ejemplo l1 y l2 miden 1 
De acuerdo con la cinemática directa para un brazo planar
$$ x = \cos(q1) + \cos(q1+q2) $$
$$ y = \sin(q1) + \sin(q1+q2) $$
$$ \begin{eqnarray} J_0 & = & \left [ \begin{array}{cc}\frac{\partial x}{\partial q1} && \frac{\partial x}{\partial q2} \\ \frac{\partial y}{\partial q1} && \frac{\partial y}{\partial q2}  \end{array} \right ]\\ & = & \left [ \begin{array}{cc}-\sin(q1) -\sin(q1+q2) && -\sin(q1+q2) \\ \cos(q1) + \cos(q1+q2) && \cos(q1+q2)  \end{array}\right ] \end{eqnarray}$$

 End-effector spatial velocity $$\nu = (v_x, v_y, v_z, \omega_x, \omega_y, \omega_z)^T$$
        is related to joint velocity by $${}^{0}\!\nu = \mathbf{J}_0(q) \dot{q}$$

robot.jacob_dot(q, qd) computes the product (6) of the temporal
derivative of the manipulator Jacobian (in the world frame) and the
joint rates $\frac{d}{dt} \dot{\mathbf{J}}(q) \dot{q}$.
This term appears in the formulation for operational space control $\ddot{x} = \mathbf{J}(q) \ddot{q} + \dot{\mathbf{J}}(q) \dot{q}$.
            


In [ ]:
def jacobian(q):
    J = np.array([ [-sin(q[0]) + -sin(q[0]+q[1]), -sin(q[0]+q[1])],
                   [cos(q[0]) + cos(q[0]+q[1]), cos(q[0]+q[1])] ])
    return J

In [ ]:
q = np.array([0,pi])
p = np.array([cos(q[0]) + cos(q[0]+q[1]),sin(q[0]) + sin(q[0]+q[1])])
q_punto = np.array([.00017, .00034]) # 1 grado y 2 grados
q_star = q + q_punto
p_punto = np.dot(jacobian(q), q_punto)
print('J=',jacobian (q))
print('p=',p)
print('p_punto',p_punto)
p_star = p[0:2] + p_punto
print('p_star',p_star)
print('directa(q_star)',robot.fkine(q_star).t)

In [ ]:
q = np.array([pi/4,pi/6])
T = robot.fkine(q)
p = T.t
print(q)
print(p)
print(T.rpy(unit='deg'))
robot.plot(q)
ax = plt.gca()
ax.view_init(elev=90, azim=-90)

In [ ]:
q_punto = np.array([.017, .034]) # 1 grado y 2 grados
q_star = q + q_punto
print(q_star)

In [ ]:
p_punto = np.dot(jacobian(q), q_punto)
print(jacobian (q))
print(p)
print(p_punto)
p_star = p[0:2] + p_punto
print(p_star)
print(robot.fkine(q_star).t)

In [ ]:
robot.plot(q_star)
ax = plt.gca()
ax.view_init(elev=90, azim=-90)

In [ ]:
T = sm.SE3(p_star[0],p_star[1],0)
q_prima, *_ = robot.ikine(T,mask=[1,1,0,0,0,0])
print(q_prima)
robot.plot(q_prima)
ax = plt.gca()
ax.view_init(elev=90, azim=-90)

In [ ]:
## print(robot.jacobe(q)) # Jacobian in end-effector frame
print(robot.jacob0(q))# Jacobian in world frame
#print(robot.jacob_dot(q,q_punto))# Jacobe dot in end-effector frame

In [ ]:
qt = rtb.tools.trajectory.jtraj(q, q_star, 50)
qt.q

In [ ]:
robot.plot(qt.q, dt=0.1)

In [ ]:
q=np.array([pi/3,-2*pi/3])
q_punto = np.array([pi/4, 0]) # 1 grado y 2 grados
q_star = q + q_punto
print(q,q_star)
qt = rtb.tools.trajectory.jtraj(q, q_star, 50)
x = np.cos(qt.q[:,0])+np.cos(qt.q[:,0]+qt.q[:,1])
y = np.sin(qt.q[:,0])+np.sin(qt.q[:,0]+qt.q[:,1])
robot.plot(qt.q, dt=0.1)
plt.plot(x,y)
ax = plt.gca()
ax.view_init(elev=90, azim=-90)

In [ ]:
x0 = np.cos(qt.q[0,0])+np.cos(qt.q[0,0]+qt.q[0,1])
y0 = np.sin(qt.q[0,0])+np.sin(qt.q[0,0]+qt.q[0,1])
for i in range(1,50):
    q_punto = qt.q[i,:] - qt.q[i-1,:]
    p_punto = np.dot(jacobian(qt.q[i-1,:]), q_punto)
    x[i]=x[i-1]+p_punto[0]
    y[i]=y[i-1]+p_punto[1]
robot.plot(qt.q, dt=0.1)
plt.plot(x,y)
ax = plt.gca()
ax.view_init(elev=90, azim=-90)

In [ ]:
y = np.linspace(0,.7,50)
x = np.sqrt(1-y**2)
for i in range(0,49):
    p_punto[0] = x[i+1] - x[i]
    p_punto[1] = y[i+1] - y[i]
    q_punto = np.dot( np.linalg.pinv(jacobian(qt.q[i,:])) , p_punto)
    qt.q[i+1]=qt.q[i]+q_punto

robot.plot(qt.q, dt=0.1)
plt.plot(x,y)
ax = plt.gca()
ax.view_init(elev=90, azim=-90)